<a href="https://colab.research.google.com/github/Ash19820/Signature-verification-model/blob/main/Signature-verification-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import librarires and modules**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 1: Install Required Libraries
!apt-get install unrar  # Install unrar to extract .rar files
!pip install wget  # Install wget to download files
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy scikit-learn pillow scikit-image tensorboard albumentations gradio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=194e9b3f4e06421c4ad8ecfd6a47775a76e46243904469a5c5f292c2f27f57f2
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
# Step 2: Import Libraries
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.tensorboard import SummaryWriter
import albumentations as A
from albumentations.pytorch import ToTensorV2
import gradio as gr
import wget

In [ ]:
# Step 4: Download the CEDAR Signature Dataset
url = "http://www.cedar.buffalo.edu/NIJ/data/signatures.rar"
output_file = "signatures.rar"

# Download the dataset
wget.download(url, output_file)

# Step 5: Extract the Dataset
!unrar x signatures.rar '/content/drive/My Drive/CEDAR signature verification/'


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from signatures.rar

Creating    /content/drive                                            OK
Creating    /content/drive/My Drive                                   OK
Creating    /content/drive/My Drive/CEDAR signature verification      OK
Creating    /content/drive/My Drive/CEDAR signature verification/signatures  OK
Creating    /content/drive/My Drive/CEDAR signature verification/signatures/full_forg  OK
Extracting  /content/drive/My Drive/CEDAR signature verification/signatures/full_forg/forgeries_10_1.png       0%  OK 
Extracting  /content/drive/My Drive/CEDAR signature verification/signatures/full_forg/forgeries_10_10.png       0%  OK 
Extracting  /content/drive/My Drive/CEDAR signature verification/signatures/full_forg/forgeries_10_11.png       0%  OK 
Extracting  /content/drive/My Drive/CEDAR signature verification/signatures/full_forg/forgeries_10_12.png       0

In [ ]:
# Configuration
config = {
    'num_epochs': 10,
    'batch_size': 32,
    'learning_rate': 0.0005,
    'margin': 1.0,
    'num_pairs_train': 900,
    'num_pairs_test': 300,
    'test_size': 0.15,
    'image_size': (220, 155),
    'threshold': 1.0,
    'log_dir': 'runs/signature_verification_experiment',
    'model_save_path': '/content/drive/My Drive/siamese_signature_model.pth',
    'dataset_prefix': '/content/drive/My Drive/CEDAR signature verification/',
    'base_org_train': 'full_org/original_%d_%d.png',
    'base_forg_train': 'full_forg/forgeries_%d_%d.png',
    'base_org_test': 'full_org/original_%d_%d.png',
    'base_forg_test': 'full_forg/forgeries_%d_%d.png',
    'num_folds': 5,  # Number of folds for K-Fold Cross-Validation
}


In [ ]:
class SignatureDataset(Dataset):
    def __init__(self, pairs, dataset_prefix, image_size):
        self.pairs = pairs
        self.dataset_prefix = dataset_prefix
        self.image_size = image_size

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]
        img1 = Image.open(os.path.join(self.dataset_prefix, img1_path)).convert('L')
        img2 = Image.open(os.path.join(self.dataset_prefix, img2_path)).convert('L')

        img1 = ImageOps.fit(img1, self.image_size, Image.ANTIALIAS)
        img2 = ImageOps.fit(img2, self.image_size, Image.ANTIALIAS)

        img1 = np.array(img1) / 255.0
        img2 = np.array(img2) / 255.0

        return torch.tensor(img1, dtype=torch.float32).unsqueeze(0), torch.tensor(img2, dtype=torch.float32).unsqueeze(0), torch.tensor(label, dtype=torch.float32)

# **Feature Extraction (Histogram of Oriented Gradients - HOG)**

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5)
        self.fc1 = nn.Linear(128 * 53 * 51, 256)
        self.fc2 = nn.Linear(256, 128)

    def forward_one(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss = (1 - label) * torch.pow(euclidean_distance, 2) + \
               (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        return torch.mean(loss)

In [ ]:
# Step 6: K-Fold Cross-Validation
def k_fold_cross_validation(model_class, dataset_class, config):
    kf = KFold(n_splits=config['num_folds'], shuffle=True)
    metrics_per_fold = []

    all_pairs = generate_signature_pairs(
        base_org=os.path.join(config['dataset_prefix'], config['base_org_train']),
        base_forg=os.path.join(config['dataset_prefix'], config['base_forg_train']),
        num_pairs=config['num_pairs_train'],
        test_size=config['test_size']
    )[0]  # Get only training pairs

    for fold, (train_idx, val_idx) in enumerate(kf.split(all_pairs)):
        print(f"Fold {fold + 1}/{config['num_folds']}")

        # Split the data into training and validation sets
        train_pairs = [all_pairs[i] for i in train_idx]
        val_pairs = [all_pairs[i] for i in val_idx]

        # Create datasets and dataloaders
        train_dataset = dataset_class(train_pairs, config['dataset_prefix'], config['image_size'])
        val_dataset = dataset_class(val_pairs, config['dataset_prefix'], config['image_size'])

        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        # Initialize model, loss, optimizer
        model = model_class().to(device)
        criterion = ContrastiveLoss(margin=config['margin']).to(device)
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

        # Train the model
        train_siamese_network(model, train_loader, criterion, optimizer, device, config, writer)

        # Evaluate the model
        evaluation_metrics = evaluate_siamese_network(model, val_loader, device, config, writer, fold)
        metrics_per_fold.append(evaluation_metrics)

    return metrics_per_fold

In [ ]:
def train_siamese_network(model, train_loader, criterion, optimizer, device, config, writer):
    model.train()
    for epoch in range(config['num_epochs']):
        total_loss = 0
        for batch_idx, (img1, img2, label) in enumerate(train_loader):
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)

            optimizer.zero_grad()
            output1, output2 = model(img1, img2)
            loss = criterion(output1, output2, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        writer.add_scalar('Loss/train', avg_loss, epoch)
        print(f"Epoch {epoch + 1}/{config['num_epochs']}, Loss: {avg_loss}")

def evaluate_siamese_network(model, val_loader, device, config, writer, fold):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for img1, img2, label in val_loader:
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            output1, output2 = model(img1, img2)
            dist = F.pairwise_distance(output1, output2)
            predicted = (dist > config['threshold']).float()
            all_labels.extend(label.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)

    writer.add_scalar('Accuracy/test', accuracy, fold)
    writer.add_scalar('Precision/test', precision, fold)
    writer.add_scalar('Recall/test', recall, fold)
    writer.add_scalar('F1_Score/test', f1, fold)

    print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1}

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Step 8: Advanced Data Augmentation
data_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Normalize(mean=(0.5,), std=(0.5,)),  # Consider adjusting mean/std
    ToTensorV2()
])

In [ ]:
def verify_signature(original, forged):
    original_image = load_and_preprocess_image(original, config['image_size'])
    forged_image = load_and_preprocess_image(forged, config['image_size'])

    model.eval()
    with torch.no_grad():
        output1, output2 = model(original_image.unsqueeze(0).to(device), forged_image.unsqueeze(0).to(device))
        dist = F.pairwise_distance(output1, output2)
        prediction = dist.item() <= config['threshold']

    return "Similar" if prediction else "Not Similar"

iface = gr.Interface(fn=verify_signature, inputs=["image", "image"], outputs="text")
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://23f43ecd1b53a97382.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install Flask
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/verify', methods=['POST'])
def verify():
    data = request.json
    original = data['original']
    forged = data['forged']

    result = verify_signature(original, forged)
    return jsonify({'result': result})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
